Imports

In [ ]:
import tkinter as tk
from tkinter import messagebox
import sqlite3
from tkcalendar import Calendar

Athlete Class

In [ ]:
class Athlete: 
# Am I actually happy with this? can some functions be moved INSIDE of the athlete class?
    def __init__(self, id, name, training_plan, current_weight, competition_weight_category, competitions_entered, private_coaching_hours, notes):
        self.id = id
        self.name = name
        self.training_plan = training_plan
        self.current_weight = current_weight
        self.competition_weight_category = competition_weight_category
        self.competitions_entered = competitions_entered
        self.private_coaching_hours = private_coaching_hours
        self.notes = notes

Data

In [ ]:
weight_categories = {
    # Category & Upper Limit
        "Heavyweight": 999,
        "Light-Heavyweight": 100,
        "Middleweight": 90,
        "Light-Middleweight": 81,
        "Lightweight": 73,
        "FlyWeight": 66}

Validations

In [ ]:
# Validate training plan entry
valid_plans = ["Beginner", "Intermediate", "Elite"]

# Validate weight category entry box for create new athlete !!!! POSSIBLE ERROR IN CODE !!!!
valid_categories = ["Heavyweight", "Light-Heavyweight", "Middleweight", "Light-Middleweight", "Lightweight", "Flyweight"]

Global Functions

In [ ]:
def open_weight_cats():
    # Create a new top-level window
    weight_cats_help_window = tk.Toplevel()  
    weight_cats_help_window.title("Weight Categories Help")  

    # Create a frame to contain the content with padding and border
    content_frame = tk.Frame(weight_cats_help_window, padx=20, pady=20, bd=2, relief="groove")
    content_frame.pack(padx=10, pady=10)

    # Header labels
    tk.Label(content_frame, text="Category", width=20, anchor="w", font=("Helvetica", 12, "bold")).grid(row=0, column=0, padx=10, pady=5)
    tk.Label(content_frame, text="Weight Limit (Kg)", width=20, anchor="e", font=("Helvetica", 12, "bold")).grid(row=0, column=1, padx=10, pady=5)

    # Adding a line separator at the top
    tk.Frame(content_frame, height=2, width=400, bg="black").grid(row=1, columnspan=2, pady=5)

    # Iterate over weight_categories dictionary (Key/Value pairs)
    for idx, (category, limit) in enumerate(weight_categories.items(), start=2):
        # Labels for category and limit
        tk.Label(content_frame, text=category, width=20, anchor="w").grid(row=idx, column=0, padx=10, pady=5, sticky="w")
        tk.Label(content_frame, text=limit, width=20, anchor="e").grid(row=idx, column=1, padx=10, pady=5, sticky="e")

Main (App) Class

In [ ]:
class App:
    def __init__(self, root):
        self.root = root
        self.root.title("N S J")   

        # Set Window Dimensions & Center on Users Screen
        width = 750
        height = 210
        screen_width = root.winfo_screenwidth()
        screen_height = root.winfo_screenheight()
        x_coordinate = (screen_width - width) // 2
        y_coordinate = (screen_height - height) // 2
        root.geometry(f"{width}x{height}+{x_coordinate}+{y_coordinate}")


        pass


    def create_table(self):
        # Create the athletes table (IF it doesn't already exist)
        pass


    def view_records(self): 
        # Create the view_records_window
        pass

        def view_selected_athlete(): # Function to view the selected athlete (open a new window populated by the athlete data selected on the view_records_window)
            pass

                    
            def update_record():
                # Function to update the record in the database
                pass

            
            def delete_record():
                # Function to delete the record from the database
                pass


    
    def create_athlete(self):
        # Create new athlete data
        pass


    def save_record(self):
        # Retrieve data from entry fields
        pass



    def competition_calendar(self): # UPDATED COMP CAL NEEDS INSERTING
        def create_entries_table(conn):
            pass

        def save_entry(conn, date, title, start_time, end_time, location, notes):
            pass

        def on_date_selected():
            pass

        def add_entry():
            pass

            def save_and_close():
                pass

        def load_entries(date):
            pass


    def __del__(self):
        # Close database connection
        self.conn.close()

Close Loop

In [ ]:
if __name__ == "__main__":
    root = tk.Tk()
    app = App(root)
    root.mainloop()